##### Celery
- python에서 사용하는 비동기 작업큐(작업툴)
- dask나 spark를 사용할 때
- message broker 가 python process를 연결해서 process를 컨트롤해 줌
- message broker(asynchronous queue) - redis
- install package
    - mac
        - `$ pip3 install celery redis`
    - windows
        - 관리자 권한으로 cmd 창에서 실행
        - celery 4.x 버전은 windows를 지원하지 않습니다.
        - `> conda install -c conda-forge celery==3.1.25`
        - `> conda install -c conda-forge redis-py`
- reference : http://www.celeryproject.org/

- python은 싱글코어이고, multi threading 사용시 각 작업이 언제 끝나는지 알 수 없지만 Celery를 사용하면 task의 상태를 계속 체크해서 언제 끝나는 지 알 수 있음

##### Redis
- mac
    - `$ brew install redis`
    - `$ redis-server`
- window
    - https://github.com/MSOpenTech/redis/releases download
        - radis-x64-3.x.msi
        - 설치후 재부팅
    - `C:\ProgramFiles\Redis> redis-cli.exe`
    - `127.0.0.1:6379> shutdown`
    - `not connected> exit`
    - `C:\ProgramFiles\Redis> redis-server.exe`

- Redis 서버 번호는 6379
- SQL과 달리 RAM을 사용해 빠른 대신 많은 정보를 담을 수 없으므로 빠른 전송이 필요한 데이터만 Redis에 올려놓고 사용

In [4]:
import redis

r = redis.StrictRedis(host='localhost', port=6379, db=0)

RAM 영역에 저장되기 때문에 컴퓨터를 끄면 삭제되므로 항상 데이터를 가져오는 코드가 필요

In [5]:
# set key and value
result = r.set('foo', 'bar')
print("set", result)

set True


get은 해당되는 데이터를 가져옴

In [6]:
# get value 
result = r.get('foo')
print("get", result)

get b'bar'


In [7]:
# decode ascii
result = result.decode('ascii')
print("get(after decode)", result)

get(after decode) bar


##### start celery
- make task.py : 파이썬 실행파일
- start celery
    - mac
        - `$ celery -A task worker`
    - windows
        - `$ celery -A task worker -l info`
- run celery

- make task.py

In [8]:
%%time
# prime_number
def prime_number(n):
    prime_list = []
    for num1 in range(1, n+1):
        is_prime = True
        for num2 in range(2, num1):
            if num1 % num2 == 0:
                is_prime = False
        if is_prime:
            prime_list.append(num1)
    return prime_list

len(prime_number(10000))

Wall time: 4.2 s


In [9]:
%%writefile task.py

from celery import Celery

BROKER_URL = 'redis://localhost:6379/0'
CELERY_RESULT_BACKEND = 'redis://localhost:6379/0'

app = Celery('task', broker=BROKER_URL, backend=CELERY_RESULT_BACKEND)

@app.task                           # decorator로 꼭 감싸야 함
def prime_number(n):                # 함수는 필요한 대로 작성
    prime_count = 0
    for num1 in range(1, n+1):
        is_prime = True
        for num2 in range(2, num1):
            if num1 % num2 == 0:
                is_prime = False
        if is_prime:
            prime_count += 1
    return prime_count

Overwriting task.py


- run celery

In [10]:
import task

In [11]:
result_1 = task.prime_number.delay(20000)
result_2 = task.prime_number.delay(10000)

In [12]:
import time

is_done_1, is_done_2 = False, False
r_1, r_2 = 0, 0
count = 0

while (not is_done_1) or (not is_done_2):   # 둘다 True가 되는 순간 빠져나옴
    is_done_1 = result_1.ready()
    is_done_2 = result_2.ready()
    time.sleep(1)
    count += 1
    
    if is_done_1:
        r_1 = result_1.get()
    if is_done_2:
        r_2 = result_2.get()
        
    print("{} sec : done1-{}, done2-{}, r_1-{}, r_2-{}".format(count, is_done_1, is_done_2, r_1, r_2))

result_1.get(), result_2.get()

1 sec : done1-False, done2-False, r_1-0, r_2-0
2 sec : done1-False, done2-False, r_1-0, r_2-0
3 sec : done1-False, done2-False, r_1-0, r_2-0
4 sec : done1-False, done2-False, r_1-0, r_2-0
5 sec : done1-False, done2-False, r_1-0, r_2-0
6 sec : done1-False, done2-False, r_1-0, r_2-0
7 sec : done1-False, done2-False, r_1-0, r_2-0
8 sec : done1-False, done2-False, r_1-0, r_2-0
9 sec : done1-False, done2-False, r_1-0, r_2-0
10 sec : done1-False, done2-False, r_1-0, r_2-0
11 sec : done1-False, done2-False, r_1-0, r_2-0
12 sec : done1-False, done2-False, r_1-0, r_2-0
13 sec : done1-False, done2-False, r_1-0, r_2-0
14 sec : done1-False, done2-False, r_1-0, r_2-0
15 sec : done1-False, done2-False, r_1-0, r_2-0
16 sec : done1-False, done2-False, r_1-0, r_2-0
17 sec : done1-False, done2-False, r_1-0, r_2-0
18 sec : done1-False, done2-False, r_1-0, r_2-0
19 sec : done1-False, done2-False, r_1-0, r_2-0
20 sec : done1-False, done2-False, r_1-0, r_2-0
21 sec : done1-False, done2-False, r_1-0, r_2-0
2

KeyboardInterrupt: 

time.sleep으로 언제 task가 완료되었는지 알 수 있음

In [13]:
29 sec : done1-True, done2-False, r_1-1230, r_2-0   # done1은 작업이 완료했으므로 r-1에 대해서는 다른 작업을 하면 됨